# 从全连接层到卷积

* a fully-connected MLP
* Because these networks are invariant to the order of the features, we could get similar results regardless of whether we preserve an order corresponding to the spatial structure of the pixels or if we permute the columns of our design matrix before fitting the MLP’s parameters.
* Preferably, we would leverage our prior knowledge that nearby pixels are typically related to each other, to build efficient models for learning from image data.

# 从全连接层到卷积

* 表格数据
    * 行对应样本，列对应特征
* 对于表格数据，我们寻找的模式可能涉及特征之间的交互，但是我们不能预先假设任何与特征交互相关的先验结构
* 此时，多层感知机可能是最好的选择


# 从全连接层到卷积

* 每张照片具有百万级像素，这意味着网络的每次输入都有一百万个维度
* 即使将隐藏层维度降低到1000，这个全连接层也将有$10^9$个参数
* 阅读“从全连接层到卷积.pdf”

课堂提问\_1：$10^9$

## 不变性

1. *平移不变性*（translation invariance）：不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的反应，即为“平移不变性”
1. *局部性*（locality）：神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远区域的关系，这就是“局部性”原则。最终，可以聚合这些局部特征，以在整个图像级别进行预测

## 多层感知机的限制

* 输入是二维图像$\mathbf{X}$，其隐藏表示$\mathbf{H}$在数学上是一个矩阵
* 

$$\begin{aligned} \left[\mathbf{H}\right]_{i, j} &= [\mathbf{U}]_{i, j} + \sum_k \sum_l[\mathsf{W}]_{i, j, k, l}  [\mathbf{X}]_{k, l}\\ &=  [\mathbf{U}]_{i, j} +
\sum_a \sum_b [\mathsf{V}]_{i, j, a, b}  [\mathbf{X}]_{i+a, j+b}.\end{aligned}$$
* 重新索引下标$(k, l)$，使$k = i+a$、$l = j+b$，由此可得$[\mathsf{V}]_{i, j, a, b} = [\mathsf{W}]_{i, j, i+a, j+b}$

### 平移不变性

* $\mathsf{V}$和$\mathbf{U}$实际上不依赖于$(i, j)$的值，即$[\mathsf{V}]_{i, j, a, b} = [\mathbf{V}]_{a, b}$
* $\mathbf{U}$是一个常数，比如$u$
* 

$$[\mathbf{H}]_{i, j} = u + \sum_a\sum_b [\mathbf{V}]_{a, b} [\mathbf{X}]_{i+a, j+b}.$$
* $[\mathbf{V}]_{a, b}$的系数比$[\mathsf{V}]_{i, j, a, b}$少很多，因为前者不再依赖于图像中的位置

### 局部性

* 在$|a|> \Delta$或$|b| > \Delta$的范围之外，我们可以设置$[\mathbf{V}]_{a, b} = 0$
* 

$$[\mathbf{H}]_{i, j} = u + \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} [\mathbf{V}]_{a, b}  [\mathbf{X}]_{i+a, j+b}.$$

### 局部性

* The price paid for this drastic reduction in parameters is that our features are now translation invariant and that our layer can only incorporate local information, when determining the value of each hidden activation.

### 局部性

* All learning depends on imposing **inductive bias**.
* But of course, if those biases do not agree with reality, e.g., if images turned out not to be translation invariant, our models might struggle even to fit our training data.

## 卷积

* $f, g: \mathbb{R}^d \to \mathbb{R}$之间的“卷积”被定义为

$$(f * g)(\mathbf{x}) = \int f(\mathbf{z}) g(\mathbf{x}-\mathbf{z}) d\mathbf{z}.$$
* *互相关*（cross-correlation）

### 通道

* 图像一般包含三个通道/三种原色（红色、绿色和蓝色）
* 前两个轴与像素的空间位置有关，而第三个轴可以看作是每个像素的多维表示
* 将$\mathsf{X}$索引为$[\mathsf{X}]_{i, j, k}$
* 卷积相应地调整为$[\mathsf{V}]_{a,b,c}$

### 通道

* 对于每一个空间位置，我们想要采用一组而不是一个隐藏表示
* 这样一组隐藏表示可以想象成一些互相堆叠的二维网格
* *通道*（channel）
* *特征映射*（feature maps） / each provides a spatialized set of learned features to the subsequent layer


### 通道

* 直观上你可以想象在靠近输入的底层，一些通道专门识别边缘，而一些通道专门识别纹理
* 

$$[\mathsf{H}]_{i,j,d} = \sum_{a = -\Delta}^{\Delta} \sum_{b = -\Delta}^{\Delta} \sum_c [\mathsf{V}]_{a, b, c, d} [\mathsf{X}]_{i+a, j+b, c}$$